In [1]:
import requests # Til at hente hjemmesider
from bs4 import BeautifulSoup # Til at trække information ud af den dowloadede html
from urllib.parse import urljoin
import json # Til at parse json
from time import sleep
#from tqdm.notebook import tqdm # Processbar
from tqdm import tqdm
from pathlib import Path # Til at arbejde med filer
import random

Det første vi gør er at finde en liste over alle kredsene.

In [2]:
stemmeseddel_url = 'https://www.dr.dk/nyheder/politik/folketingsvalg/din-stemmeseddel'
stemmeseddel_req = requests.get(stemmeseddel_url) # Hent siden
stemmeseddel_soup = BeautifulSoup(stemmeseddel_req.text) # Ved at sætte .text får man html koden fra siden

Hiv links til kredsene ud af siden:

In [3]:
kreds_urls_ugly = stemmeseddel_soup('a', class_='AccordionGrid_link__cGkec') # extracting the links 'a' we want. By looking at the page with the inspect feature, we see that all the links we're interested in are classified with AccordionGrid_link__cGkec.
kreds_urls = list(map(lambda x: urljoin(stemmeseddel_url, x['href']), kreds_urls_ugly)) # ? Combining the stemmeseddel and kreds url... somehow.

#kreds_urls = list(map(lambda x: urljoin(stemmeseddel_url, x['href']), stemmeseddel_soup('a', class_='AccordionGrid_link__cGkec')))

På hver kreds-side er der en liste over alle kandidaterne, så alle de lister finder vi:

In [4]:
def kreds_kandidater(kreds_url):
    req = requests.get(kreds_url) # Hent kreds-siden
    soup = BeautifulSoup(req.text) # Strukturer html, så vi kan søge i den
    
    kandidater_soup = soup.find('script', id='__NEXT_DATA__').text # don't get this part
    kandidater_json = json.loads(kandidater_soup) # Træk json ud af siden
    kandidater_data = kandidater_json['props']['pageProps']['smallConstituencyCandidatesByPartyCode'] # udtræk kun kandidat info fra json
    
    # constituents_data[0] er alle kandiater fra første parti i kredsen (i JSON format). I kreds_urls[0] - Rønne, er det socialkrabaterne, Lea, Steen og Lars.
    
    # Find alle url'er
    kandidat_urls = []
    for parti in kandidater_data: # gå i gennem alle partier i kredsen
        for kandidat in list(parti.values())[0]['candidates']: # for alle kandidater i partiet
            kandidat_urls.append(kandidat['urlKey']) # gem kandidatens URL
    return kandidat_urls

In [5]:
kandidat_urls = []
for kreds_url in tqdm(kreds_urls): # gå igennem alle kredse
    kandidat_urls += kreds_kandidater(kreds_url) # i kreds, gem kandidat URL
    sleep(1)

100%|███████████████████████████████████████████| 92/92 [03:04<00:00,  2.01s/it]


Vi gemmer alle links til kandidaterne i en fil, så vi kan bruge det senere.

In [6]:
#open("candidate_urls.txt", "x")
urlfile = Path('kandidat_urls') # path() 
with urlfile.open('w') as f: # with automatically closes the file again, avoiding us closing it again manually
                             # the 'w' allows ud to write in the file
                             # with open as f, the file is in the variable f
    for url in set(kandidat_urls): # candidate_urls indeholder alle kandidaterne flere gange fordi vi ikke har begrænset os til storkredsene, så vi fjerner dubletter ved at bruge 'set'
        f.write(url + '\n')

Nu kan vi hente data om hver enkelt kandidat:

In [7]:
kandidat_baseurl = 'https://www.dr.dk/nyheder/politik/folketingsvalg/din-stemmeseddel/kandidater/'

In [8]:
for kandidat_url in tqdm(set(kandidat_urls)):
    kandidat_fil = Path(f'kandidater/{kandidat_url}.json') # Vi vil gemme data i en fil
    if kandidat_fil.exists(): continue #skip resten af loop

    req = requests.get(kandidat_baseurl + kandidat_url) # Hent siden
    soup = BeautifulSoup(req.text)
    kandidat_data = soup.find('script', id='__NEXT_DATA__').text # Træk data ud
    kandidat_fil.write_text(kandidat_data) # Gem det write_text laver filen
    sleep(random.random() * 8) # Vi sover lidt for ikke at spamme serveren (det er nok ikke så vigtigt givet størrelsen på DR, men høfligst)

100%|███████████████████████████████████████| 1014/1014 [16:24<00:00,  1.03it/s]
